# Ejemplo de uso

In [1]:
import mib_v2_3_2 as mb

## Modelo de un problema

$P(ABC) = P(A)P(B|A)P(C|AB)$.

### Probabilidades

<table width="90%">
    <tr>
        <td valign="top">
            <table width="50%">
              <tr>
                <th> $A$</th>
                <th><p style="text-align:center;"> $P(A)$</p></th>
              </tr>
              <tr>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
              </tr>
            </table>
        </td>
        <td valign="top">
            <table width="60%">
              <tr>
                <td style="height:1px;"> &nbsp;</td>
                <td colspan="2"><p style="text-align:center;">$P$($B$|$A$)</p></td>
              </tr>
              <tr>
                <td>$A$</td>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
                <td><p style="text-align:center;">$0.8$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
              </tr>
            </table>        
        </td>
        <td valign="top">
            <table width="90%">
              <tr>
                <td style="height:1px;"> &nbsp;</td>
                <td colspan="2"><p style="text-align:center;">$P$($C$|$AB$)</p></td>
              </tr>
              <tr>
                <td>$AB$</td>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$2$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$00$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
                <td><p style="text-align:center;">$0.8$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$01$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
                <td><p style="text-align:center;">$0.5$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$10$</p></td>
                <td><p style="text-align:center;">$0.4$</p></td>
                <td><p style="text-align:center;">$0.5$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$11$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
              </tr>
            </table>        
        </td>
    </tr>
</table>

## Creación de datos

### Declaración de diccionarios

In [2]:
# Tablas
# {(variables independientes): {variables: p}}
# {(A,): P(A)}
dA = {
    (0,): 0.3,
    (1,): 0.7
}

# {(A,):{(B,): P(B|A)}}
dB_A = {
    (0,):{(0,): 0.2, (1,): 0.8},
    (1,):{(0,): 0.3, (1,): 0.7}
}

# {(A,B):{(C,): P(C|A,B)}}
dC_AB = {
    (0,0): {(0,): 0.1, (1,): 0.8, (2,): 0.1},
    (0,1): {(0,): 0.3, (1,): 0.5, (2,): 0.2},
    (1,0): {(0,): 0.4, (1,): 0.5, (2,): 0.1},
    (1,1): {(0,): 0.1, (1,): 0.7, (2,): 0.2}
    }


In [ ]:
TA = [0.3, 0.7]
TB_A = [
    [0.2, 0.8],
    [0.3, 0.7]
]
TC_AB = [
    [0.1, 0.8, 0.1],
    [0.3, 0.5, 0.2], 
    [0.1, 0.7, 0.2]
]



for t in range(100000):
    xi = []
    for i in range(len(T)):
        r = 0
        for j in range(len(x)):
            r += (x[j] * T[j][i])
        xi.append(r)
    x = xi.copy()
print(x)

### Declaración de variables

In [3]:
A = mb.Var(name='A', values=set([0,1]))
B = mb.Var(name='B', values=set([0,1]))
C = mb.Var(name='C', values=set([0,1,2]))

### Declaración de distribuciones 

In [4]:
# P(A)
PA = mb.Distrib(table=dA, vars=(A,))
# P(B|A)
PB_A = mb.Distrib(table=dB_A, vars=(B,), parents=(A,))
# P(C|AB)
PC_AB = mb.Distrib(table=dC_AB, vars=(C,), parents=(A, B))

## Creación de la pregunta

In [5]:
# Probabilidad conjunta (Especificación)
PABC = mb.Specification(set([A,B,C]), (PA, PB_A, PC_AB))
# Generador de consultas (Pregunta)
QABC = mb.Question(PABC)

## Creación de consultas

### Distribuciones

#### De una variable

In [6]:
PB = QABC.DistributionQuery(set([B]))
PB.table

{(0,): 0.27, (1,): 0.7789999999999999}

In [14]:
PB = QABC.DistributionQuery(set([B]), aproximation=True)
PB.table

{(0,): 0.2738, (1,): 0.7339}

#### Conjuntas

In [7]:
PAB = QABC.DistributionQuery(set([A,B]))
PAB.table

{(0, 0): 0.06, (0, 1): 0.21, (1, 0): 0.24, (1, 1): 0.5389999999999999}

In [15]:
PAB = QABC.DistributionQuery(set([A,B]), aproximation=True)
PAB.table

{(0, 0): 0.06, (0, 1): 0.2104, (1, 0): 0.2449, (1, 1): 0.4898}

#### Condicional

In [16]:
PA_B = QABC.DistributionQuery(set([A]), set([B]))
parents_names = tuple([v.name for v in PA_B.parents])
vars_names = tuple([v.name for v in PA_B.vars])
print(parents_names, vars_names)
PA_B.table

('B',) ('A',)


{(0,): {(0,): 0.2222222222222222, (1,): 0.7777777777777777},
 (1,): {(0,): 0.3080872913992298, (1,): 0.6919127086007701}}

In [18]:
PA_B = QABC.DistributionQuery(set([A]), set([B]), aproximation=True)
parents_names = tuple([v.name for v in PA_B.parents])
vars_names = tuple([v.name for v in PA_B.vars])
print(parents_names, vars_names)
PA_B.table

('B',) ('A',)


{(0,): {(0,): 0.2180673824509441, (1,): 0.7815188926300037},
 (1,): {(0,): 0.3219733406623609, (1,): 0.6685210312075984}}

### Consulta de probabilidades

In [9]:
# P(A=0|C=0)
p = QABC.Query((A,), (C,), (0,), (0,))
p

0.3

In [10]:
# P(A=0,B=0)
p = QABC.Query(vars=(A,B), vars_values=(0,0))
p

0.06

### Consultas de valores más probables

In [11]:
#P(A|B=0)
p = QABC.Query(vars=(A,), indep=(B,), indep_values=(0,))
p

(('A',), (1,), 0.7777777777777777)

In [12]:
#P(A=0|B)
p = QABC.Query(vars=(A,), indep=(B,), vars_values=(0,))
p

(('B',), (1,), 0.3080872913992298)

In [13]:
#P(AB)
p = QABC.Query(vars=(A,B))
p

(('A', 'B'), (1, 1), 0.5389999999999999)